(NOTE: all of the following locations are referenced from leads home directory: /autofs/cluster/animal/scan_data/leads). This script first unfoils the dicom folder to identify any repeat MPRAGEs in ./LEADS. It will unpack the dicoms into an mgz that will go into the ./recon_nip/RECON_3T/ and ./recon_nip/RECON_FLAIR/ respective folders and start the initial recons for the two pipelines. The scaninfo is unpacked from the dicoms and put into ./recon_nip/RECON_NOTES folder and scan notes downloaded from LONI and put into ./spreadsheets/ which will also be put into the database in recon_nip/RECON_NOTES/? As you update and QC the recons (and rerun them as you edit -- don't forget to include the FLAIR flag! Otherwise it will not apply. But you can always add the flair later as well as it is part of autorecon3 (postediting stage).

REQUIREMENTS FOR THIS PIPELINE
the recon output will be named e.g. "unedit.FS6_02". After manually editing please rename this recon folder to "edit.FS6_02" and start another recon manually. Then the next time this script is run, it will record the appropriate information into the databases.
the following notes need to be updated upon each batch download so this info can be piped:

        details:
        MOST IMPORTANT: once a new download has been initiated, load the CSV download file into
        /autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/ (REQUIRED!!!!)
        if it is a new image collection on loni, it will create a separate CSV file, 
        otherwise will replace old one with concatenated data of that image collection. So archive
        any csvs that are old versions of downloaded collections. But the code also does this automatically.
        
        Also, occasionally download Mayo_ADRIL_MRI_Quality_date.csv and AMYELG and xxxx into ./spreadsheets to
        update database before analyses -- (e.g. we need the AMY info for this).
        
        Download and save to the following locations:
        Mayo_ADRIL_MRI_Quality_date.csv to /autofs/cluster/animal/scan_data/leads/spreadsheets/MRIQUALITY
        this file will replace the old one (concatenates all data on loni about QC).

If any files need to be re-run / re-processed, delete the files in the folder. This pipeline does not overwrite anything.

In [1]:
# TO DO
# send a recon job- just do for ones that do not have output?
# decide if I want to have the option to re-run edits manually of implement here?
# way to automate csv download

In [2]:
# import modules
import io, os, sys, types # needed
import glob # needed
from nipype.pipeline.engine import Workflow, Node, MapNode # needed
from nipype.interfaces.utility import Function, IdentityInterface
import nipype.interfaces.io as nio
import nipype.pipeline.engine as pe
from nipype.interfaces.freesurfer import MRIConvert
from nipype.interfaces.freesurfer import ReconAll
from nipype import config
import pandas as pd
import re
import shutil
import pathlib
import pydicom
from pydicom.tag import Tag

In [3]:
# Clean and update spreadsheets

# these do not concatenate, must do this
downloadsdir = '/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/'

# vertically concatenate all csvs
downloadlist = glob.glob(downloadsdir+'*.csv')
downloadlist.remove('/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/combined_downloads.csv')

# vertically concatenate all csvs
combined_csv = pd.concat( [ pd.read_csv(f) for f in downloadlist ] , sort=False)

# drop all non-MPRAGES, sort dataframe by subject column, drop all duplicates
combined_csv_index = combined_csv.Description.str.contains('Accelerated Sagittal MPRAGE|Sagittal 3D Accelerated MPRAGE', regex=True)
combined_csv['keep'] = combined_csv_index
combined_csv = combined_csv[combined_csv.keep == True]

combined_csv = combined_csv.sort_values(by=['Downloaded'])
combined_csv = combined_csv.drop_duplicates(['Image Data ID'], keep='last')

del combined_csv['keep']

# # save combined download file
combined_csv.to_csv("/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/combined_downloads.csv", index=False,)

# # these download already concatenating all sessions ; use latest
qualitydir = '/autofs/cluster/animal/scan_data/leads/spreadsheets/MRIQUALITY/'

list_of_qualityfiles = glob.glob(qualitydir+'*.csv')
MRIQUALITY = max(list_of_qualityfiles, key=os.path.getctime)

In [4]:
def scan(subject):
    subsessions = glob.glob(dicomdir+subject+'/Accelerated_Sagittal_MPRAGE/*/*/')
    mprage_name = 'Accelerated_Sagittal_MPRAGE'
    if subsessions == []:
        subsessions = glob.glob(dicomdir+subject+'/Sagittal_3D_Accelerated_MPRAGE/*/*/')
        mprage_name = 'Sagittal_3D_Accelerated_MPRAGE'
    repeat_tag = '-'
    print(mprage_name)
    print(subsessions)
    for num in range(len(subsessions)):
        # look to see if more than one session on the same date
        # then look to see if more than one date (or both)
        parentfolder = subsessions[num].split('/')[8]
        filename = os.listdir(subsessions[num])[0] # dicom name to extract date
        #extract date from dicom:
        ds = pydicom.read_file(subsessions[num]+'/'+filename)
        #print(ds)
        try:
            date = str(ds[0x08, 0x22].value)
        except(KeyError):
            date = str(ds[0x08, 0x21].value)
        print(date)
        if num == len(subsessions)-1:
            namedate = dicomdir+subject+'_'+date
            os.rename(dicomdir+subject, namedate)
        else:
            namedateplus = dicomdir+subject+'_'+date+repeat_tag+'/'+mprage_name+'/'+parentfolder
            pathlib.Path(namedateplus).mkdir(parents=True, exist_ok=True)
            shutil.move(subsessions[num], namedateplus)
            if not os.listdir(dicomdir+subject+'/'+mprage_name+'/'+parentfolder):
                os.rmdir(dicomdir+subject+'/'+mprage_name+'/'+parentfolder)
            repeat_tag = repeat_tag+'-'

In [5]:
# specify variables
leadsdir = '/cluster/animal/scan_data/leads/'
os.chdir(leadsdir)
dicomdir = "/cluster/animal/scan_data/leads/LEADS/"
unpacklog = "/autofs/cluster/animal/scan_data/leads/recon_nip/SCAN_NOTES/unpack.log"
recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/RECON_FLAIR/'
recondir_3t = '/autofs/cluster/animal/scan_data/leads/recon_nip/RECON_3T/'
recondir_edit = '/autofs/cluster/animal/scan_data/leads/recon_nip/RECON_FLAIR_EDITED/'
folders = [x for x in os.listdir(dicomdir) if not x.startswith(".")]
subjlist = [f for f in os.listdir(dicomdir) if (("_") not in f) and (not f.startswith('.') and ("duplicate" not in f))]

# wipe clean batch.recon.list
open('/autofs/cluster/animal/scan_data/leads/recon_nip/batch.recon.list', 'w').close()

# unwravel all subjects with multiple sessions and rename to include date
for sub in subjlist:
    scan(sub)

# now just make a list of subjects by ID (this will be the input to the nodes)
# will define dicom path within node
sh_dicomlist = [f for f in os.listdir(dicomdir) if (("_" in f) and ("REPEAT_RUNS" not in f))]

# define workflow
leads_workflow = Workflow(name='leads_workflow') #, base_dir = '/autofs/cluster/animal/scan_data/leads/recon_nip/') # add  base_dir='/shared_fs/nipype_scratch'

# configure to stop on first crash
cfg = dict(execution={'stop_on_first_crash': True})
config.update_config(cfg)

Sagittal_3D_Accelerated_MPRAGE
[]


In [6]:
#sh_dicomlist = [x for x in os.listdir('/autofs/cluster/animal/scan_data/leads/recon_nip/RECON_FLAIR') if x.startswith("LDS")] 

In [7]:
# for debugging
sh_dicomlist = ['LDS0180106_20190621']
#['LDS0220154_20190812','LDS3600140_20190813'] #,'

#['LDS0730144_20190801','LDS0730150_20190731','LDS3600119_20190807','LDS0180153_20190806']


In [8]:
# TEST NODE : PASSSWORDS

def credentials(): # combined with find_dicom
    import getpass
    USER = getpass.getuser()
    print('Please enter your PASSWORD for launchpad access: ')
    PASS= getpass.getpass()
    return USER, PASS

PASSWORDS = pe.Node(Function(input_names=["user", "pw"],
                         output_names=["USER","PASS"], # actual dicom (redundant to create unpacking node visualization)
                         function=credentials),
                        name='PASSWORDS')

In [9]:
# NODE : CREATEDIR
def createdir(val, USER, PASS):
    import os
    import re
    import glob
    import pydicom
    from pydicom.tag import Tag
    val = val.split('/')[-1]
    dicomdir = "/autofs/cluster/animal/scan_data/leads/LEADS/"
    pipelines = ['RECON_3T', 'RECON_FLAIR']
    for pipe in pipelines:
        recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'+pipe+'/' 
        reconpath = recondir+val+'/'
        imgpath = reconpath+'mri/orig/'
        if not glob.glob(reconpath + '/**/*mri', recursive=True): # changed recondir to reconpath
            os.makedirs(imgpath) # edited this part!
    dumplocation = imgpath+'001.mgz'
    flairdumplocation = imgpath+'FLAIR.mgz'
    subject = val.split('_')[0]
    mprage_names = ['Accelerated_Sagittal_MPRAGE','Sagittal_3D_Accelerated_MPRAGE'] # nomenclature differs
    for name in mprage_names:
        try:
            MPRAGE_path = glob.glob(dicomdir+val+'/'+name+'/*/*')[0]
            pickdicom = glob.glob(dicomdir+val+'/'+name+'/*/*/*')[0]
        except(IndexError):
            pass
    pickflair = glob.glob(dicomdir+val+'/Sagittal_3D_FLAIR/*/*/*')[0]
    ds = pydicom.read_file(pickdicom)
    try:
        date = str(ds[0x08, 0x22].value)
    except(KeyError):
        date = str(ds[0x08, 0x21].value)
    sessionid = MPRAGE_path.split("/")[-1]
    return reconpath, MPRAGE_path, pickdicom, dumplocation, recondir, USER, PASS, imgpath, date, flairdumplocation, pickflair
        
CREATEDIR = pe.Node(Function(input_names=["val", "USER", "PASS"],
                         output_names=["createdir_out1","createdir_out2", "createdir_out3", "createdir_out4", "createdir_out5", "USER", "PASS", "createdir_out6", "date", "flairdumplocation","pickflair"], # actual dicom (redundant to create unpacking node visualization)
                         function=createdir),
                        name='CREATEDIR')

In [10]:
# NODE : IMPORT_LONI_INFO

def import_loni_notes(dicomname, date, subjectdir):
    import pandas as pd
    import glob
    import os
    import re
    # download info
    download_df = pd.read_csv('/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/combined_downloads.csv')
    recon_dir = '/autofs/cluster/animal/scan_data/leads/recon_nip/SCAN_NOTES/'
    mgh_subs = '/autofs/cluster/animal/scan_data/leads/spreadsheets/IDENTIFICATION/MGH_SUBJECTS.tsv'
    notes_dir = recon_dir+subjectdir.split('/')[-2]
    dicom = dicomname.split("/")[-1]
    subid = dicom.split("_")[1]
    imageid = dicom.split("_")[12][1:-4]
    scannotes_df = pd.read_csv(notes_dir+'/scannotes.csv')
    try:
        download_date = download_df.loc[download_df['Image Data ID'] == float(imageid), 'Downloaded'].values[0]
    except(IndexError):
        pass
    except(UnboundLocalError):
        raise Exception('LONI download information for one or more included subjects cannot be located. Please update spreadsheet in LONI_DOWNLOADS. For more information about this, read comments in beginning of this script.')
    # loni notes
    qualitydir = '/autofs/cluster/animal/scan_data/leads/spreadsheets/MRIQUALITY/'
    list_qc_files = glob.glob(qualitydir+'*.csv')
    MRIQUALITY = max(list_qc_files, key=os.path.getctime)
    # these download already concatenating all sessions ; use latest
    quality_df = pd.read_csv(MRIQUALITY)
    try:
        loni_overallpass = quality_df.loc[quality_df['loni_image'] == float(imageid), 'study_overallpass'].values[0]
        if loni_overallpass == 1:
            qc_pass = '1'
        elif loni_overallpass == 4:
            qc_pass = '0'
        else:
            qc_pass = ''
        study_comments = quality_df.loc[quality_df['loni_image'] == float(imageid), 'study_comments'].values[0]
        study_protocol_comment = quality_df.loc[quality_df['loni_image'] == float(imageid), 'study_protocol_comment'].values[0]
        protocol_comments = quality_df.loc[quality_df['loni_image'] == float(imageid), 'protocol_comments'].values[0]
        series_comments = quality_df.loc[quality_df['loni_image'] == float(imageid), 'series_comments'].values[0]
        series_quality = quality_df.loc[quality_df['loni_image'] == float(imageid), 'series_quality'].values[0]# if 3, needs review; if 2 it is ok
        if series_quality == 2:
            s_quality = 'Scan quality is acceptable according to MAYO. '
        elif series_quality == 3:
            s_quality = 'Scan quality is questionable according to MAYO and needs review. '
        elif series_quality == 4:
            s_quality = 'Scan quality is poor according to MAYO and needs review. '
        else:
            s_quality = 'No scan quality data recorded from MAYO. '
        study_rescan_requested = quality_df.loc[quality_df['loni_image'] == float(imageid), 'study_rescan_requested'].values[0]
        if study_rescan_requested == 'TRUE':
            rescan_requested = ' Study rescan has been requested. '
        else:
            rescan_requested = '. No study rescans have been requested. '

        # delete duplicates within list, delete nans
        comments_list = [s_quality,str(study_comments),str(study_protocol_comment),str(series_comments),str(protocol_comments),rescan_requested]
        cleanedList = [x for x in comments_list if (x != 'nan')]
        concat_comments = ''.join(cleanedList)+" QC_pass from original site is "+qc_pass+" ."
        xnat_upload = '0'
    except(IndexError):
        if subid[0:6] == 'LDS360':   # if its MGH data # line 57
            mgh_df = pd.read_csv(mgh_subs,index_col=False, sep='\t')
            concat_comments = mgh_df.loc[mgh_df['leads_sessionid'] == subid+'_'+date, 'notes'].values[0]
            xnat_upload = mgh_df.loc[mgh_df['leads_sessionid'] == subid+'_'+date, 'XNAT_upload'].values[0]
            if str(mgh_df.loc[mgh_df['leads_sessionid'] == subid+'_'+date, 'notes'].values[0]) == 'nan':
                concat_comments = 'No comments from MAYO. '
            qc_pass = "No data."
        else:
            concat_comments = 'No comments from MAYO. '
            qc_pass = 'No data.'
            xnat_upload = '0'
            
    # add recon path by taking newest edit folder (if exists)
    try:
        edit_dir = '/autofs/cluster/animal/scan_data/leads/recon_nip/RECON_FLAIR_EDITED/'+subjectdir.split('/')[-2]
        edit_folders = [f for f in os.listdir(edit_dir) if f.startswith("edit.")]
        ext_folders = [edit_dir + s for s in edit_folders]
        try:
            recon_folder = max(ext_folders, key=os.path.getctime)
        except(ValueError):
            recon_folder = ''
    except(FileNotFoundError):
        recon_folder = ''
        
    scannotes_df.loc[scannotes_df.index[0], 'leads_id'] = subid+'_'+date
    scannotes_df.loc[scannotes_df.index[0], 'mayo_overallpass'] = qc_pass
    scannotes_df.loc[scannotes_df.index[0], 'mayo_notes'] = concat_comments
    scannotes_df.loc[scannotes_df.index[0], 'download_date'] = download_date
    scannotes_df.to_csv(notes_dir+'/scannotes.csv', index=False)
    return dicomname, subjectdir, imageid
    
IMPORT_LONI_INFO = pe.Node(Function(input_names=["dicomname", "date", "subjectdir"],
                        output_names = ["dicomname", "subjectdir", "imageid"], 
                        function=import_loni_notes),
                        name='IMPORT_LONI_INFO')


In [11]:
# NODE : PREPARE_4_REDCAP
# note that QC notes and status are manually recorded in scannotes

def prepare_redcap(dicomname, subjectdir, imageid, pickdicom, fsversion):
    import pandas as pd
    import pydicom
    from pydicom.tag import Tag
    import csv
    subject = subjectdir.split("/")[-1]
    convert_sex = '/autofs/cluster/animal/scan_data/leads/spreadsheets/IDENTIFICATION/DEMOGRAPHIC_IDS.csv'
    demo_form = '/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/combined_downloads.csv'
    site_conversion = '/autofs/cluster/animal/scan_data/leads/spreadsheets/IDENTIFICATION/SITE_IDS.csv'
    notes_dir = subjectdir.replace('RECON_FLAIR','SCAN_NOTES')
    scannotes = pd.read_csv(notes_dir+'scannotes.csv')
    download_df = pd.read_csv(demo_form)
    reader = csv.reader(open(convert_sex))
    ds = pydicom.read_file(pickdicom)
    d={}
    for row in reader:
        d[row[0]]=row[1:][0]
    sex = str(ds[0x10,0x40].value)
    SEX = d.get(sex)
    try:
        AQ_DATE = ds[0x08,0x22].value
    except(KeyError):
        AQ_DATE = ds[0x08,0x21].value
    age = ds[0x00101010].value
    try:
        AGE = str(int(age[:-1]))
    except(ValueError):
        AGE = ''
    
    try:
        SITE = str(ds[0x08, 0x80].value) #Institution Name
    except(KeyError):
        d={}
        reader = csv.reader(open(site_conversion))
        for row in reader:
            d[row[0]]=row[1:][0]
        t = subject[3:6]
        SITE = d.get(t)    

    dicom_path = pickdicom.strip(pickdicom.split('/')[-1]) # or use scaninfo ?
    GROUP = download_df.loc[download_df['Image Data ID'] == float(imageid), 'Group'].values[0]
    GEN_NOTES = ''
    RECON_PATH = '' #decide which recon to use
    FS_VERSION = fsversion
    
    #Can be found in scannotes:
    DN_DATE = scannotes.loc[scannotes.index[0], 'download_date']
    acq_notes = scannotes.loc[scannotes.index[0], 'mayo_notes']


PREPARE_4_REDCAP = pe.Node(Function(input_names=["dicomname","subjectdir", "imageid", "pickdicom", "fsversion"],
                        function=prepare_redcap),
                          name='PREPARE_4_REDCAP')

In [12]:
# NODE : UNPACK

def unpack(subjectdir, MPRAGE_path, pickdicom):
    from os import system
    from os import path
    from os import makedirs
    import csv
    import pandas as pd
    import os.path
    import pydicom
    notesdir = subjectdir.replace('RECON_FLAIR',"SCAN_NOTES")
    ## add header and save (if 8 columns, it's old type ; if 9 its new :: new can unpack GE and Philips and Siemens)
    #old_unpack_sys_header = ['run/series number','protocol','error status','number of columns','number of rows','number of slices','number of frames','DICOM file']
    new_unpack_sys_header = ['run/series number','protocol','echo time','repetition time', 'flip angle', 'unknown', 'In-Plane-Phase-Encoding Dir','pixel bandwidth','DICOM file', 'manufacturer']
    if not path.exists(notesdir):
        makedirs(notesdir)
    if not os.path.isfile(notesdir+'scan.info'): 
        #cmdstring = 'unpacksdcmdir -src %s -targ %s -scanonly %s/scan.info' % (MPRAGE_path, notesdir, notesdir)
        cmdstring = 'dcmunpack -src %s -scanonly %s/scan.info' % (MPRAGE_path,notesdir)
        system(cmdstring)
    if not os.path.isfile(notesdir+'scaninfo.csv'):
        ds = pydicom.read_file(pickdicom)
        MANU = str(ds[0x08, 0x70].value) # look at dicom manufacturer field
        with open(notesdir+'/scan.info', 'r') as in_file:
            for line in in_file:
                editline = line.split()
                editline.append(MANU)
                with open(notesdir+'/scaninfo.csv', 'w') as result:
                    wr = csv.writer(result, dialect='excel')
                    wr.writerow(editline)
                result.close()
            in_file.close()
        scaninfo = pd.read_csv(notesdir+'/scaninfo.csv', names=new_unpack_sys_header)
        scaninfo.to_csv(notesdir+'/scaninfo.csv', index=False)
    scan_info = notesdir+'/scaninfo.csv'
    
    subname = notesdir.split('/')[-2]
    return subname, subjectdir, scan_info, MPRAGE_path, pickdicom

UNPACK = pe.Node(Function(input_names=["subjectdir","MPRAGE_path", "pickdicom"],
                         output_names=["unpack_out1","unpack_out2", "unpack_out3", "unpack_out4","unpack_out5"], # actual dicom (redundant to create unpacking node visualization)
                         function=unpack),
                        name='UNPACK')


In [13]:
# NODE CHECK DICOM INFO

def check_info(subname, subjectdir, scan_info, MPRAGE_path, pickdicom):
    import pandas as pd
    import pydicom
    notesdir = subjectdir.replace('RECON_FLAIR',"SCAN_NOTES")
    scaninfo = pd.read_csv(notesdir+'scaninfo.csv')
    # check to make sure TR is filled out (for some scans MGH function dcmunpack does not work)
    if scaninfo.loc[0, "echo time"] == "unknown":
        ds = pydicom.read_file(pickdicom)
        series = ds[0x52009229].value
        tmp = str(series[0]).split("\n")
        for el in tmp:
            if "(0018, 0080)" in el: # repetition time
                TR = el.split("\"")[1]
            elif "(0018, 1312)" in el: # phase encoding dir
                encoding_dir = el.split("\'")[1]
            elif "(0018, 1314)" in el: #flip angle
                flip_angle = el.split("\"")[1]
            elif "(0018, 0095)" in el:
                pixel_band = el.split("\"")[1]
        image = ds[0x52009230].value
        tmp2 = str(image[0]).split("\n")
        for el in tmp2:
            if "(0018, 9082)" in el:  # echo time
                echo_time = str(el).split("FD: ")[1]
        scaninfo['echo time'] = echo_time
        scaninfo['repetition time'] = TR
        scaninfo['flip angle'] = flip_angle
        scaninfo['In-Plane-Phase-Encoding Dir'] = encoding_dir
        scaninfo['pixel bandwidth'] = pixel_band
        scaninfo.to_csv(notesdir+'/scaninfo.csv', index=False)
    else:
        pass
    return subname, subjectdir, scan_info

CHECKINFO = pe.Node(Function(input_names=["subname", "subjectdir", "scan_info", "MPRAGE_path", "pickdicom"],
                         output_names=["check_out1","check_out2", "check_out3"], # actual dicom (redundant to create unpacking node visualization)
                         function=check_info),
                        name='CHECKINFO')

In [14]:
# (first option) # # NODE : CONVERT2MGZ (only runs if .mgz is not available)

def convert_dicom(in_file, out_file, reconpath):
    import os
    import glob
    from os import system
    #import time # just see if this works if waits
    # check for a file called 001.mgz
    if not glob.glob(reconpath + '/**/*001.mgz', recursive=True):
        cmdstring = 'mri_convert %s %s' % (in_file, out_file)
        system(cmdstring)
        complete = 1
    else:
        complete = 1

    return complete

CONVERT2MGZ = pe.Node(Function(input_names=["in_file", "out_file", "reconpath"],
                         output_names=["out_file"],
                         function=convert_dicom),
                        name='CONVERT2MGZ')

In [15]:
def convert_flair(pickflair, flairdumplocation, reconpath, out_file):
    import os
    import glob
    from os import system
    import shutil
    reconpath_3t = reconpath.replace('RECON_FLAIR','RECON_3T')

    # put back after copying all flairs ad hoc
    if not glob.glob(reconpath + '/**/*FLAIR.mgz', recursive=True):
        cmdstring = 'mri_convert %s %s' % (pickflair, flairdumplocation)
        system(cmdstring)
        complete = 1
    else:
        complete = 1  
    if not glob.glob(reconpath_3t + '/**/*001.mgz', recursive=True):
        for root, dirs, files in os.walk(reconpath): 
            for file in files:  
                if file == '001.mgz': 
                    shutil.copyfile(root+'/'+str(file), reconpath_3t+'/mri/orig/001.mgz')
                    complete = 1
    else:
        complete = 1
    return complete

CONVERTFLAIR = pe.Node(Function(input_names=["pickflair", "flairdumplocation", "reconpath", "out_file"],
                         output_names=["out_file"],
                         function=convert_flair),
                        name='CONVERTFLAIR')

In [16]:
# NODE SCAN_AND_LOG
# note: decided to add this afterward precaution to increase efficiency because there are few errors
# and want to run the unpack and convert2mgz in parallel)

def scan_and_log(subjectdir, scan_info, mgz, reconfolder, subname):
    import re
    import os
    import pandas as pd
    notesdir = subjectdir.replace('RECON_FLAIR',"SCAN_NOTES")
    # load in the scaninfo file
    dicomdir = "/cluster/animal/scan_data/leads/LEADS/"
    scaninfo = pd.read_csv(scan_info)
    with open('/autofs/cluster/animal/scan_data/leads/recon_nip/batch.recon.list', "a") as bfile:
        bfile.write(subname)
    with open('/autofs/cluster/animal/scan_data/leads/recon_nip/SCAN_NOTES/unpack.log', "a") as ufile:
        ufile.write(subname)
    # should I makea scannotes? (will add info after recon)
    Elements = {'leads_id': [''],'mayo_notes': [''],'mayo_overallpass': [''], 'download_date':['']}
    df = pd.DataFrame(Elements, columns= ['leads_id','mayo_notes', 'mayo_overallpass', 'download_date'])
    df.to_csv(notesdir+'/scannotes.csv',index=False)
    return subjectdir, subname

SCAN_AND_LOG = pe.Node(Function(input_names=["subjectdir","scan_info",'mgz', 'reconfolder', 'subname'],
                         output_names=["subjectdir", "subname"],
                         function=scan_and_log),
                        name='SCAN_AND_LOG')

In [17]:
# NODE RECON_JOB

def recon_job(subjectname, USER, PASS): # add in username, pass, and subjectname
    # add condition :: run this only is FS_XX, or scripts does not exist!!
    import os
    import glob
    from paramiko import SSHClient
    analyses_pipes = ['RECON_FLAIR','RECON_3T']
    for pipeline in analyses_pipes:
        reconpath = '/autofs/cluster/animal/scan_data/leads/recon_nip/'+pipeline
        if not glob.glob('/autofs/cluster/animal/scan_data/leads/recon_nip/'+pipeline+'/'+subjectname + '/**/*scripts', recursive=True):
            host="launchpad"
            user=USER
            pw=PASS
            client=SSHClient()
            client.load_system_host_keys()
            client.connect(host,username=user,password=pw, look_for_keys=False)
            tmpstr = '(cd /autofs/cluster/animal/scan_data/leads/analyses_nip/%s; setenv p %s ; ./batch.recon.sh)' % (pipeline, subjectname)
            stdin, stdout, stderr = client.exec_command(tmpstr)
            err = "stderr: ", stderr.readlines()
            out = "pwd: ", stdout.readlines()
            if len(err) < 1:
                warning = '0'
            else:
                warning = '1'
            with open(reconpath+'/log_nip.txt','a') as outf:
                outf.write(tmpstr+'\n')
        else:
            err = ""
            out = ""
            warning = "na"
            
    return err, out, warning, subjectname

RECON_JOB = pe.Node(Function(input_names=["subjectname","USER", "PASS"], 
                        output_names=[ 'err', 'out', 'warning','subjectname'],
                         function=recon_job),
                        name='RECON_JOB')


In [18]:
# NODE: GATHER_FS_DETAILS (this part only after recon is done)

def gather_FS_details(subjectname): # add in username, pass, and subjectname
    import csv
    import os
    import pandas as pd
    pipelines = ['RECON_FLAIR','RECON_3T']
    recon_pending = []
    recon_name = []
    for pip in pipelines:
        recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'+pip+'/'
        # if you can access the status (complete) and in directory
        try: 
            with open(recondir+subjectname+'/scripts/recon-all.done') as f:
                first_line = f.readline()
            if first_line == '1\n':
                recon_pending.append(1) # catch recons that are done processing but with errors
            else:
                recon_pending.append(0)
                
            # obtain FS version
            versionfile = open(recondir+subjectname+'/scripts/build-stamp.txt', 'r')
            versionstring = versionfile.read()
            version = versionstring.split('-')
            result = [i for i in version if i.startswith('v')][0]
            long = result[1:]

            #obtain short verison of long
            size = len(long)
            x = 0
            while x ==0:
                if (long[-1] == '0') or (long[-1] == '.'): # shave off any . or 0s from the end of version number.
                    long = long[:-1]
                else:
                    x =1
            vlabel = 'FS'+long

            # obtain run number
            notesdir = recondir.replace(pip,"SCAN_NOTES")
#             with open(notesdir+subjectname+'/scaninfo.csv','r') as f:
#                 reader = csv.reader(f)
#                 scan_list = list(reader)
#                 runstring = scan_list[0][0] # run
            scaninfo = pd.read_csv(notesdir+subjectname+'/scaninfo.csv')    # newly added !!
            runstring = str(scaninfo.loc[0,'run/series number'])          # newly added !!
            if len(runstring) == 1:
                runstring = '0'+runstring
            recon_name.append(vlabel+'_'+runstring)
        
        #else: # otherwise incomplete, not run yet, or already moved (could have errors though)
        except(FileNotFoundError):
            recon_pending.append(1)
            recon_name.append('')
            long = '' 
    return subjectname, recon_name, recon_pending, long, pipelines 

FS_DETAILS = pe.Node(Function(input_names=["subjectname"], 
                        output_names=[ 'subjectname', 'recon_name', 'recon_pending', 'long', 'pipelines'],
                         function=gather_FS_details),
                        name='FS_DETAILS')

In [19]:
# NODE : MAKE_ORIG_FOLDER

def create_orig_folder(subjectname, recon_name, recon_pending, pipelines):
    import os
    import shutil
    freesurfer_dirs = ['mri', 'stats', 'tmp', 'trash', 'touch', 'label', 'surf', 'scripts']
    for idx, pip in enumerate(pipelines):
        recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'+pip+'/'
        if recon_pending[idx] == 0:
            # move all subfolders into this recon_name folder
            for fsdir in freesurfer_dirs:
                if os.path.isdir(recondir+subjectname+'/'+fsdir):
                    shutil.move(recondir+subjectname+'/'+fsdir, recondir+subjectname+'/'+recon_name[idx]+'/'+fsdir) # does this create FS6_02?
        else:
            print(subjectname+" files moves already, or not yet prepared.")
    return subjectname, recon_name, recondir, recon_pending, pipelines

MAKE_ORIGINAL_DIR = pe.Node(Function(input_names=["subjectname", "recon_name", "recon_pending", "pipelines"], 
                        output_names=['subjectname', 'recon_name', 'recondir', 'recon_pending', 'pipelines'],
                         function=create_orig_folder),
                        name='MAKE_ORIGINAL_DIR')



In [20]:
# NODE : PREPARE_MANEDITS

def preparing_manedits(subjectname, recon_name, recon_pending, pipelines):
    import shutil
    import pathlib
    import os
    for idx, pip in enumerate([pipelines[0]]): # just doing edits for FLAIR pipeline
        recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'+pip+'/'
        analysesdir = '/autofs/cluster/animal/scan_data/leads/analyses_nip/'+pip+'/'
        if recon_pending[idx] == 0: # otherwise dir already created or not ready)
            recon_name2 = 'unedit.'+recon_name[idx] # overwrite recon_name2 through iteration
            shutil.copytree(recondir+subjectname+'/'+recon_name[idx], recondir+subjectname+'/'+recon_name2)
            shutil.copyfile(recondir+subjectname+'/'+recon_name2+'/mri/brain.finalsurfs.mgz', recondir+subjectname+'/'+recon_name2+'/mri/brain.finalsurfs.manedit.mgz')
    return subjectname, recon_name, recon_pending, pipelines
        
PREPARE_MANEDITS = pe.Node(Function(input_names=['subjectname', 'recon_name', 'recon_pending', 'pipelines'], 
                        output_names=['subjectname', 'recon_name', 'recon_pending', 'pipelines'],
                         function=preparing_manedits),
                        name='PREPARE_MANEDITS')

In [21]:
# NODE : EXTRACT_RECON_DETAILS AND PUT INTO RECON_NOTES.CSV

def make_pandas(subjectname, recon_name, recon_pending, pipelines, long):
    import pandas as pd
    import numpy as np
    import os
    status, pbsjob, command, subdir, recon_path = ([["",""],["",""]] for i in range(5))
    final = ['FS', 'edit']
    
    for idx, pip in enumerate(pipelines):
        subjectdir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'+pip+'/'+subjectname+'/'
        notes = pd.read_csv('/autofs/cluster/animal/scan_data/leads/recon_nip/'+pip+'/recon_notes.csv',index_col= 'LEADS_ID').replace(np.nan, '', regex=True)
        current_subjects = notes.index.values
        if subjectname not in current_subjects: #subject is not in notes:
            notes_cols = ['LEADS_ID'] + list(notes)
            initialize_fields = [[''] * len(notes_cols)]
            new_df = pd.DataFrame(columns=notes_cols, data=initialize_fields)
            new_df.at[[0],'LEADS_ID'] = subjectname 
            # set other values too
            new_df = new_df.set_index(['LEADS_ID'])
            notes = pd.concat([new_df, notes])
            notes = notes.sort_values('LEADS_ID', axis=0, ascending=True)
            notes.to_csv('/autofs/cluster/animal/scan_data/leads/recon_nip/'+pip+'/recon_notes.csv') #,index_col= 'LEADS_ID')
        # update the relevant fields (for new and old subjects
        
        #for init in final:
        for versidx, init in enumerate(final):
            folders = [subjectdir+x for x in os.listdir(subjectdir) if x.startswith(init)] # this means recon is running or done

            try: 
                statusfile = max(folders, key=os.path.getmtime)+'/scripts/recon-all.done'
                with open(statusfile, 'r') as fh:
                    num_lines = sum(1 for line in open(statusfile))
                    if num_lines < 2:
                        qcstatus = 'error' # check for recon errors
                        recon_path[idx][versidx] = ''
                    else:
                        for line in fh:
                            if line.startswith("CMDARGS"):
                                com = line.replace("CMDARGS","recon-all") # initial recon command
                                command[idx][versidx] = com[:-1]
                        qcstatus = 'complete'
                        recon_path[idx][versidx] = max(folders, key=os.path.getmtime)
            except(ValueError): # the folder does not exist (need especially for edit.)
                qcstatus = '' # subject is still running
                command[idx][versidx] = ''
                recon_path[idx][versidx] = ''
            except(FileNotFoundError): # the file does not exist but the folder does
                qcstatus = 'inprogress' # subject is still running
                command[idx][versidx] = ''
                recon_path[idx][versidx] = ''

            try:
                envfile = max(folders, key=os.path.getmtime)+'/scripts/recon-all.env'
                with open(envfile, 'r') as fw:
                    for line in fw:
                        if line.startswith("PBS_JOBNAME"):
                            pbsjob[idx][versidx] = line.replace("PBS_JOBNAME=","").rstrip()
                        elif line.startswith('SUBJECTS_DIR'):
                            subdir[idx][versidx] = line.replace("SUBJECTS_DIR=","").rstrip()
            except(ValueError): # status (still running or has not begun running)
                qcstatus = '' # overwrite recon did not start
                pbsjob[idx][versidx] = ''
                subdir[idx][versidx] = ''
            except(FileNotFoundError): # the file does not exist
                qcstatus = '' # overwrite variable recon did not start
                pbsjob[idx][versidx] = ''
                subdir[idx][versidx] = ''            
            status[idx][versidx] = qcstatus

        # for pip set variables
        notes = pd.read_csv('/autofs/cluster/animal/scan_data/leads/recon_nip/'+pip+'/recon_notes.csv').replace(np.nan, '', regex=True)
        notes.loc[notes['LEADS_ID'] == subjectname, 'FS_INITIAL_SUBDIR'] = subdir[idx][0]
        notes.loc[notes['LEADS_ID'] == subjectname, 'FS_POSTEDIT_SUBDIR'] = subdir[idx][1]
        notes.loc[notes['LEADS_ID'] == subjectname, 'FS_INITIAL_COMMAND'] = command[idx][0]
        notes.loc[notes['LEADS_ID'] == subjectname, 'FS_POSTEDIT_COMMAND'] = command[idx][1]
        notes.loc[notes['LEADS_ID'] == subjectname, 'PBSJOB_INITIAL'] = pbsjob[idx][0]
        notes.loc[notes['LEADS_ID'] == subjectname, 'PBSJOB_EDIT'] = pbsjob[idx][1]
        notes.loc[notes['LEADS_ID'] == subjectname, 'FS_VERSION'] = long # this does not work unless before folder is copied?
        notes.loc[notes['LEADS_ID'] == subjectname, 'RECON_PATH'] = ''
        notes.loc[notes['LEADS_ID'] == subjectname, 'EDITOR'] = 'RJE'    
        if pip == 'RECON_FLAIR':
            notes.loc[notes['LEADS_ID'] == subjectname, 'RECON_PATH'] = recon_path[idx][1] # recon edit.FS..
            notes.loc[notes['LEADS_ID'] == subjectname, 'STATUS_FINAL'] = status[idx][1]
        elif pip == 'RECON_3T':
            notes.loc[notes['LEADS_ID'] == subjectname, 'RECON_PATH'] = recon_path[idx][0] # recon FS..
            notes.loc[notes['LEADS_ID'] == subjectname, 'STATUS_FINAL'] = status[idx][0]
        notes.to_csv('/autofs/cluster/animal/scan_data/leads/recon_nip/'+pip+'/recon_notes.csv', index=False)
    return subjectname
            
INITIALIZE_SUBJECT = pe.Node(Function(input_names=['subjectname', 'recon_name', 'recon_pending', 'pipelines', 'long'], 
                        output_names=['subjectname'],
                         function=make_pandas),
                        name='INITIALIZE_SUBJECT')

In [22]:
# # # NODE : INFOSOURCE
INFOSOURCE = Node(IdentityInterface(fields=['subject_name'], mandatory_inputs=False),
                  name="INFOSOURCE")

INFOSOURCE.iterables = ('subject_name', sh_dicomlist)

# NODE : SELECTFILES
#templates = dict(dicom=sh_dicomlist[0])    ## THIS WORKED!
templates = {
    "dicom": "{subject_name}" 
    }
SELECTFILES = Node(nio.SelectFiles(templates, base_directory=dicomdir),
                   name="SELECTFILES")

# NODE : DATASINK
DATASINK = Node(nio.DataSink(base_directory=leadsdir,
                container='recon_nip'),
                name="DATASINK")

In [23]:
# Connect all nodes (including INFOSOURCE, SELECTFILES, and DATASINK) to workflow

leads_workflow.connect([(INFOSOURCE, SELECTFILES, [('subject_name', 'subject_name')]),
                (SELECTFILES, CREATEDIR, [('dicom', 'val')]),
                (PASSWORDS, CREATEDIR, [('USER', 'USER')]),
                (PASSWORDS, CREATEDIR, [('PASS', 'PASS')]), 
                (CREATEDIR, IMPORT_LONI_INFO, [('date', 'date')]),
                (CREATEDIR, IMPORT_LONI_INFO, [('createdir_out3', 'dicomname')]),
                (CREATEDIR, IMPORT_LONI_INFO, [('createdir_out1', 'subjectdir')]),     # need actual subjectdir name (in case of repeats)
                (IMPORT_LONI_INFO, PREPARE_4_REDCAP, [('dicomname', 'dicomname')]),
                (IMPORT_LONI_INFO, PREPARE_4_REDCAP, [('subjectdir', 'subjectdir')]),
                (IMPORT_LONI_INFO, PREPARE_4_REDCAP, [('imageid', 'imageid')]),
                (CREATEDIR, PREPARE_4_REDCAP, [('createdir_out3', 'pickdicom')]),
                (CREATEDIR, UNPACK, [('createdir_out1', 'subjectdir')]),
                 (CREATEDIR, UNPACK, [('createdir_out2', 'MPRAGE_path')]),
                (CREATEDIR, UNPACK, [('createdir_out3', 'pickdicom')]),
                 (CREATEDIR, CONVERT2MGZ, [('createdir_out3', 'in_file')]),
                 (CREATEDIR, CONVERT2MGZ, [('createdir_out4', 'out_file')]),
                (CREATEDIR, CONVERT2MGZ, [('createdir_out1', 'reconpath')]),
                (CONVERT2MGZ, CONVERTFLAIR, [('out_file', 'out_file')]), # added 
                (CREATEDIR, CONVERTFLAIR, [('flairdumplocation', 'flairdumplocation')]),
                 (CREATEDIR, CONVERTFLAIR, [('pickflair', 'pickflair')]),
                (CREATEDIR, CONVERTFLAIR, [('createdir_out1', 'reconpath')]),
                (CONVERTFLAIR, SCAN_AND_LOG, [('out_file', 'mgz')]), #changed
                (CREATEDIR, SCAN_AND_LOG, [('createdir_out5', 'reconfolder')]),
                (UNPACK, CHECKINFO, [('unpack_out1', 'subname')]),
                (UNPACK, CHECKINFO, [('unpack_out2', 'subjectdir')]),
                (UNPACK, CHECKINFO, [('unpack_out3', 'scan_info')]),
                (UNPACK, CHECKINFO, [('unpack_out4', 'MPRAGE_path')]),
                (UNPACK, CHECKINFO, [('unpack_out5', 'pickdicom')]),      
                (CHECKINFO, SCAN_AND_LOG, [('check_out1', 'subname')]),
                (CHECKINFO, SCAN_AND_LOG, [('check_out2', 'subjectdir')]),
                (CHECKINFO, SCAN_AND_LOG, [('check_out3', 'scan_info')]),
                (CREATEDIR, RECON_JOB, [('USER','USER')]),
                (CREATEDIR, RECON_JOB, [('PASS','PASS')]),
                (SCAN_AND_LOG, RECON_JOB, [('subname','subjectname')]), 
                (RECON_JOB, FS_DETAILS, [('subjectname','subjectname')]),
                (FS_DETAILS, MAKE_ORIGINAL_DIR, [('subjectname','subjectname')]), 
                (FS_DETAILS, MAKE_ORIGINAL_DIR, [('recon_pending','recon_pending')]), 
                (FS_DETAILS, MAKE_ORIGINAL_DIR, [('recon_name','recon_name')]), 
                (FS_DETAILS, MAKE_ORIGINAL_DIR, [('pipelines','pipelines')]), 
                (MAKE_ORIGINAL_DIR, PREPARE_MANEDITS, [('subjectname','subjectname')]), 
                (MAKE_ORIGINAL_DIR, PREPARE_MANEDITS, [('recon_name','recon_name')]), 
                (MAKE_ORIGINAL_DIR, PREPARE_MANEDITS, [('recon_pending','recon_pending')]),
                (MAKE_ORIGINAL_DIR, PREPARE_MANEDITS, [('pipelines','pipelines')]),
                (FS_DETAILS, INITIALIZE_SUBJECT, [('long','long')]), 
                (PREPARE_MANEDITS, INITIALIZE_SUBJECT, [('subjectname','subjectname')]), 
                (PREPARE_MANEDITS, INITIALIZE_SUBJECT, [('recon_name','recon_name')]), 
                (PREPARE_MANEDITS, INITIALIZE_SUBJECT, [('recon_pending','recon_pending')]),
                (PREPARE_MANEDITS, INITIALIZE_SUBJECT, [('pipelines','pipelines')]),                           
                (FS_DETAILS, PREPARE_4_REDCAP, [('long','fsversion')]), 
                (PREPARE_MANEDITS, DATASINK, [('subjectname','backup')])  # backup folder?
                 ])

In [24]:
# Execute workflow in sequential way
# leads_workflow.run(run(plugin='MultiProc', plugin_args={'n_procs' : 2})
leads_workflow.run()

leads_workflow.write_graph(dotfilename='/autofs/cluster/animal/scan_data/leads/leads_workflow/workflow_graph.dot',graph2use='flat')

190830-10:39:10,4 nipype.workflow INFO:
	 Workflow leads_workflow settings: ['check', 'execution', 'logging', 'monitoring']
190830-10:39:10,51 nipype.workflow INFO:
	 Running serially.
190830-10:39:10,53 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.SELECTFILES" in "/tmp/tmpl3di3tqp/leads_workflow/_subject_name_LDS0180106_20190621/SELECTFILES".
190830-10:39:10,57 nipype.workflow INFO:
	 [Node] Running "SELECTFILES" ("nipype.interfaces.io.SelectFiles")
190830-10:39:10,66 nipype.workflow INFO:
	 [Node] Finished "leads_workflow.SELECTFILES".
190830-10:39:10,68 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.PASSWORDS" in "/tmp/tmp_sp7qcfl/leads_workflow/PASSWORDS".
190830-10:39:10,72 nipype.workflow INFO:
	 [Node] Running "PASSWORDS" ("nipype.interfaces.utility.wrappers.Function")
Please enter your PASSWORD for launchpad access: 
········
190830-10:39:13,936 nipype.workflow INFO:
	 [Node] Finished "leads_workflow.PASSWORDS".
190830-10:39:13,938 nipype.workflow INFO:


'/autofs/cluster/animal/scan_data/leads/leads_workflow/workflow_graph.png'

In [25]:
#leads_workflow.write_graph(graph2use='flat')

In [26]:
# import dickerson_database

# # # specific upload mr function
# # dickerson_database.dropbox.upload_mr.upload_mr(subject_id, date, t1_path, dry=True)

# # # general dropbox interface
# # dickerson_dropbox = dickerson_database.dropbox.DickersonLab()
# print(dickerson_dropbox.exists('/Dickerson lab/0_Subject Data'))

# print(dickerson_dropbox.list('/Dickerson lab'))
# #   ['0_Subject Data',
# #  'Abstracts',
# #  'Administrative',
# #  'Autopsy Reports',
# #  ...]
# # dickerson_dropbox.upload('/tmp/foo.nii.gz', '/Dickerson lab/foo.nii.gz')